# Machine learning with TensorFlow: hidden and softmax

Date: August 21-23, 2018

Neural network: hidden and softmax

Classification of human body motion:
- walking
- sitting down
- turning right while walking
- turning left while walking

In [669]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sg
import serial
import time
import random

## Preparing data set for training CNN

In [670]:
SAMPLE_NUMS = 260  # per sec
SAMPLING_RATE = 80.0  # Hz
GYRO_RESOLUTION = 250.0 / 32768.0
ACCEL_RESOLUTION = 2.0 / 32768.0

TRAINING_SET = [['./20180821_walking_straight.csv', 0],
               ['./20180821_sitting.csv', 1],
               ['./20180821_turning_left.csv', 2],
               ['./20180821_turning_right.csv', 3],                
               ['./20180823_walking_straight.csv', 0],
               ['./20180823_sitting.csv', 1],
               ['./20180823_turning_left.csv', 2],
               ['./20180823_turning_right.csv', 3]]

TEST_SET = [['./20180822_walking_straight.csv', 0],
            ['./20180822_sitting.csv', 1],
            ['./20180822_turning_left.csv', 2],
            ['./20180822_turning_right.csv', 3]]

TIME_INTERVAL = 260.0 / SAMPLING_RATE  # sec

MEASUREMENTS = 8
RECORDS = 260

In [671]:
gres = lambda v: v * GYRO_RESOLUTION
ares = lambda v: v * ACCEL_RESOLUTION
to_time = lambda v: v / SAMPLING_RATE

def conv(df, lpf=False):
    df[['gx', 'gy', 'gz']] = df[['gx', 'gy', 'gz']].apply(gres)
    df[['ax', 'ay', 'az']] = df[['ax', 'ay', 'az']].apply(ares)
    if lpf:
        CUTOFF = 10.0
        b, a = sg.butter(5, CUTOFF/SAMPLING_RATE, btype='low')
        df[['ax', 'ay', 'az']] = df[['ax', 'ay', 'az']].apply(lambda row: sg.lfilter(b, a, row))
        CUTOFF = 10.0
        b, a = sg.butter(5, CUTOFF/SAMPLING_RATE, btype='low')
        df[['gx', 'gy', 'gz']] = df[['gx', 'gy', 'gz']].apply(lambda row: sg.lfilter(b, a, row))
    df[['cnt']] = df[['cnt']].apply(to_time)
    df.set_index('cnt', drop=True, inplace=True)
    # measurements = df.tail(1).iloc[0,0] + 1
    # return measurements

In [672]:
training_set = []
test_set = []

def label(l):
    ll = [0,0,0,0]
    ll[l] = 1
    return ll

for file, l in TRAINING_SET:
    df = pd.read_csv(file, dtype=np.int16)
    conv(df, lpf=True)
    for i in range(MEASUREMENTS):
        training_set.append([df[df['id']==i], label(l)])
        
for file, l in TEST_SET:
    df = pd.read_csv(file, dtype=np.int16)
    conv(df, lpf=True)
    for i in range(MEASUREMENTS):
        test_set.append([df[df['id']==i], label(l)])
    
random.shuffle(training_set)
random.shuffle(test_set)

In [673]:
# use accel x axis and gyro z axis values for classification of human body motion
train_x, train_t = [], []
test_x, test_t = [], []
for df, label in training_set:
    values = np.concatenate((df['ax'].values, df['ay'].values, df['az'].values,
                             df['gx'].values, df['gy'].values, df['gz'].values),
                            axis=None)
    train_x.append(values)
    train_t.append(label)
for df, label in test_set:
    values = np.concatenate((df['ax'].values, df['ay'].values, df['az'].values,
                             df['gx'].values, df['gy'].values, df['gz'].values),
                            axis=None)
    test_x.append(values)
    test_t.append(label)

## TensorFlow

In [674]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20180823)
tf.set_random_seed(20180823)

In [675]:
num_samples = SAMPLE_NUMS * 6
num_units = 64
num_classes = 4

x = tf.placeholder(tf.float32, [None, num_samples])

w1 = tf.Variable(tf.truncated_normal([num_samples, num_units]))
b1 = tf.Variable(tf.zeros([num_units]))
hidden1 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)

w0 = tf.Variable(tf.zeros([num_units, num_classes]))
b0 = tf.Variable(tf.zeros([num_classes]))
p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0)

In [676]:
t = tf.placeholder(tf.float32, [None, num_classify])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer().minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [677]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [678]:
i = 0
for _ in range(2000):
    i += 1
    sess.run(train_step, feed_dict={x:train_x, t:train_t})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:test_x, t:test_t})
        print('Step: {}, Loss: {}, Accuracy: {}'.format(i, loss_val, acc_val))

Step: 100, Loss: 18.7894287109375, Accuracy: 0.875
Step: 200, Loss: 12.07949447631836, Accuracy: 0.9375
Step: 300, Loss: 9.543143272399902, Accuracy: 0.96875
Step: 400, Loss: 8.124893188476562, Accuracy: 0.96875
Step: 500, Loss: 7.3722758293151855, Accuracy: 0.9375
Step: 600, Loss: 7.091572284698486, Accuracy: 0.90625
Step: 700, Loss: 6.780047416687012, Accuracy: 0.90625
Step: 800, Loss: 6.571867942810059, Accuracy: 0.90625
Step: 900, Loss: 6.410127639770508, Accuracy: 0.90625
Step: 1000, Loss: 6.289735317230225, Accuracy: 0.90625
Step: 1100, Loss: 6.199483871459961, Accuracy: 0.90625
Step: 1200, Loss: 6.131598949432373, Accuracy: 0.90625
Step: 1300, Loss: 6.065449237823486, Accuracy: 0.90625
Step: 1400, Loss: 6.023398399353027, Accuracy: 0.90625
Step: 1500, Loss: 5.97792387008667, Accuracy: 0.90625
Step: 1600, Loss: 7.009959697723389, Accuracy: 0.90625
Step: 1700, Loss: 7.008603096008301, Accuracy: 0.90625
Step: 1800, Loss: 7.02732515335083, Accuracy: 0.90625
Step: 1900, Loss: 7.06791

In [679]:
p_test = sess.run(p, feed_dict={x:test_x})
(p_test*100).astype(int)[:10]

array([[ 7, 90,  0,  1],
       [66, 27,  5,  0],
       [ 5,  0,  0, 94],
       [ 0,  0, 99,  0],
       [ 1,  0,  0, 98],
       [92,  0,  7,  0],
       [ 3,  0,  0, 96],
       [ 0,  0, 99,  0],
       [ 0, 99,  0,  0],
       [ 0,  0, 99,  0]])

In [680]:
test_t[:10]

[[0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0]]

In [681]:
sess.close()